# Get data from Einnsyn

Gets records from einnsyn for a specific search. The provided function helps you search between two dates for a specific organisation, but modify the data object to your needs.

## How to find the id of the public organization / requst json
* Go to einnsyn.no
* Choose your organisantion from the list
* Inspect the network traffic
* Find the call to https://einnsyn.no/api/result
* Check the request json, you'll find your chosen organizsation under `arkivskaperTransitive`
* If multiple organisations, make them an array
* If used for a more custom query, build it in einnsyn and take the whole request json an paste it in data

In [1]:
# Notebook config, enviroment and logging

import os
import sys

import logging
from dotenv import load_dotenv

import pakkenellik.config as conf
from pakkenellik.log import logger

# Add module path and load config
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Autoreload extension
if "autoreload" not in get_ipython().extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Load the .env file into local env
load_dotenv()

# Create the config
config = conf.Config(module_path)

# Enable logging and crank up log level to DEBUG.
# This is particularly useful when developing code in your project module and using it from a notebook.
logger.setLevel(logging.INFO)

In [2]:
## Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

In [3]:
## Specific imports
import json
import requests
from time import sleep
from datetime import datetime

In [4]:
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 1, 15)

In [5]:
def search_einnsyn(from_date, to_date, virksomhet, offset=0):

    data = {
        "size": 50,
        "offset": offset,
        "aggregations": {
            "contentTypes": "type",
            "virksomheter": "arkivskaperTransitive",
        },
        "appliedFilters": [
            {
                "to": f"{to_date.strftime('%Y-%m-%d')}||/d",
                "from": f"{from_date.strftime('%Y-%m-%d')}||/d",
                "fieldName": "dokumentetsDato",
                "type": "rangeQueryFilter",
            },
            {
                "fieldName": "type",
                "fieldValue": ["JournalpostForMøte"],
                "type": "notQueryFilter",
            },
            {
                "fieldName": "arkivskaperTransitive",
                "fieldValue": [virksomhet],
                "type": "postQueryFilter",
            },
        ],
        "sort": {"fieldName": "_score", "order": "ASC"},
    }

    headers = {
        "Content-Type": "application/json;charset=UTF-8",
        "Accept": "application/json",
        "Accept-Language": "no,nb;q=0.9,nn;q=0.8,en;q=0.7,de;q=0.6,nb-NO;q=0.5,en-US;q=0.4,fr;q=0.3,nl;q=0.2,da;q=0.1,es;q=0.1,sv;q=0.1,is;q=0.1,und;q=0.1",
    }
    payload = json.dumps(data)
    res = requests.post("https://einnsyn.no/api/result", data=payload, headers=headers)

    return res.json()

In [6]:
d = search_einnsyn(
    start_date,
    end_date,
    "http://data.einnsyn.no/virksomhet/oepeksport_content_provider_93",
    0,
)

In [7]:
dfs = []
for offset in range(0, d["hitCount"], 50):
    sleep(0.5)
    d = search_einnsyn(
        start_date,
        end_date,
        "http://data.einnsyn.no/virksomhet/oepeksport_content_provider_93",
        offset,
    )
    _df = pd.json_normalize(d["searchHits"])
    dfs.append(_df)
    print(offset)
df_all = pd.concat(dfs)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300


In [8]:
df_all.head(10)

,score,id,source.parent.offentligTittel_SENSITIV,source.parent.saksaar,source.parent.saksnummerGenerert,source.parent.offentligTittel,source.parent.saksnummer,source.parent.id,source.parent.type,source.parent.sakssekvensnummer,source.publisertDato,source.journaldato,source.type,source.arkivskaperSorteringNavn,source.mottaker,source.standardDato,source.dokumentetsDato,source.skjerming.tilgangsrestriksjon,source.skjerming.id,source.skjerming.type,source.skjerming.skjermingshjemmel,source.id,source.avsender,source.arkivskaperNavn,source.dokumentbeskrivelse,source.offentligTittel_SENSITIV,source.mottaker_SENSITIV,source.saksnummerGenerert,source.offentligTittel,source.avsender_SENSITIV,source.korrespondansepart,source.sorteringstype,source.arkivskaperTransitive,source.journalpostnummer,source.journalsekvensnummer,source.journalposttype,source.journalaar,source.arkivskaper
0,0.0,http://data.einnsyn.no/noark4/Journalpost--972...,Frankrike - Helse,2016,"[2016/13617, 13617/2016, 16/13617, 13617/16]",Frankrike - Helse,2016/13617,http://data.einnsyn.no/noark4/Saksmappe--97241...,[Saksmappe],13617,2023-01-10T09:27:48.243Z,2023-01-03,[Journalpost],Utenriksdepartementet,[Helse- og omsorgsdepartementet],2023-01-03T00:00:00,2022-01-11,http://www.arkivverket.no/standarder/noark5/ar...,b0,[Skjerming],-,http://data.einnsyn.no/noark4/Journalpost--972...,[],[Utenriksdepartementet],[],Frankrike. Svar på instruks. Anmodning om skje...,[Helse- og omsorgsdepartementet],"[2016/13617-136, 13617/2016-136, 16/13617-136,...",Frankrike. Svar på instruks. Anmodning om skje...,[],[{'korrespondansepartNavn': 'Helse- og omsorgs...,utgående,[http://data.einnsyn.no/virksomhet/oepeksport_...,136,148220,http://www.arkivverket.no/standarder/noark5/ar...,2022,http://data.einnsyn.no/virksomhet/oepeksport_c...
1,0.0,http://data.einnsyn.no/noark4/Journalpost--972...,Afghanistan 2021 / 2022 – Opphold / asyl i Nor...,2022,"[2022/8093, 8093/2022, 22/8093, 8093/22]",Afghanistan 2021 / 2022 – Opphold / asyl i Nor...,2022/8093,http://data.einnsyn.no/noark4/Saksmappe--97241...,[Saksmappe],8093,2023-01-06T09:04:56.836Z,2022-12-30,[Journalpost],Utenriksdepartementet,[*],2022-12-30T00:00:00,2022-01-14,http://www.arkivverket.no/standarder/noark5/ar...,b0,[Skjerming],-,http://data.einnsyn.no/noark4/Journalpost--972...,[],[Utenriksdepartementet],[],Reply - request for your kindly,[Screened],"[2022/8093-441, 8093/2022-441, 22/8093-441, 80...",Reply - request for your kindly,[],"[{'korrespondansepartNavn': '*', 'id': 'http:/...",utgående,[http://data.einnsyn.no/virksomhet/oepeksport_...,441,148472,http://www.arkivverket.no/standarder/noark5/ar...,2022,http://data.einnsyn.no/virksomhet/oepeksport_c...
2,0.0,http://data.einnsyn.no/noark4/Journalpost--972...,Afghanistan 2021 / 2022 – Opphold / asyl i Nor...,2022,"[2022/8093, 8093/2022, 22/8093, 8093/22]",Afghanistan 2021 / 2022 – Opphold / asyl i Nor...,2022/8093,http://data.einnsyn.no/noark4/Saksmappe--97241...,[Saksmappe],8093,2023-01-06T09:04:56.836Z,2022-12-30,[Journalpost],Utenriksdepartementet,[*],2022-12-30T00:00:00,2022-01-12,http://www.arkivverket.no/standarder/noark5/ar...,b0,[Skjerming],-,http://data.einnsyn.no/noark4/Journalpost--972...,[],[Utenriksdepartementet],[],Reply - help my sister please,[Screened],"[2022/8093-440, 8093/2022-440, 22/8093-440, 80...",Reply - help my sister please,[],"[{'korrespondansepartNavn': '*', 'id': 'http:/...",utgående,[http://data.einnsyn.no/virksomhet/oepeksport_...,440,148470,http://www.arkivverket.no/standarder/noark5/ar...,2022,http://data.einnsyn.no/virksomhet/oepeksport_c...
3,0.0,http://data.einnsyn.no/noark4/Journalpost--972...,Østersjørådet CBSS Norsk formannskap 2021-2022,2020,"[2020/18252, 18252/2020, 20/18252, 18252/20]",Østersjørådet CBSS Norsk formannskap 2021-2022,2020/18252,http://data.einnsyn.no/noark4/Saksmappe--97241...,[Saksmappe],18252,2023-01-03T15:02:55.858Z,2022-12-27,[Journalpost],Utenriksdepartementet,"[Viken fylkeskommune, Kunnskapsdepartementet, ...",2022-12-27T00:0

In [18]:
df_all["sak_doknr"] = df_all["source.saksnummerGenerert"].str[2]
df_all["dokdato_str"] = df_all["source.dokumentetsDato"]
df_all["dokdato"] = pd.to_datetime(df_all.dokdato_str, format="%Y-%m-%d")
df_all["journaldato_einnsyn"] = pd.to_datetime(
    df_all["source.journaldato"], format="%Y-%m-%d"
)
df_all["publisertdato_einnsyn"] = pd.to_datetime(df_all["source.publisertDato"])
df_all["doktittel"] = df_all["source.offentligTittel"]
df_all["saktittel"] = df_all["source.parent.offentligTittel"]
df_all["journaltype"] = df_all["source.type"].str[0]

In [19]:
df_all.head(10)

,score,id,source.parent.offentligTittel_SENSITIV,source.parent.saksaar,source.parent.saksnummerGenerert,source.parent.offentligTittel,source.parent.saksnummer,source.parent.id,source.parent.type,source.parent.sakssekvensnummer,source.publisertDato,source.journaldato,source.type,source.arkivskaperSorteringNavn,source.mottaker,source.standardDato,source.dokumentetsDato,source.skjerming.tilgangsrestriksjon,source.skjerming.id,source.skjerming.type,source.skjerming.skjermingshjemmel,source.id,source.avsender,source.arkivskaperNavn,source.dokumentbeskrivelse,source.offentligTittel_SENSITIV,source.mottaker_SENSITIV,source.saksnummerGenerert,source.offentligTittel,source.avsender_SENSITIV,source.korrespondansepart,source.arkivskaperTransitive,source.journalpostnummer,source.journalsekvensnummer,source.journalposttype,source.journalaar,source.arkivskaper,sak_doknr,dokdato_str,dokdato,journaldato_einnsyn,publisertdato_einnsyn,doktittel,saktittel,journaltype
0,0.0,http://data.einnsyn.no/noark4/Journalpost--972...,Tsjekkia - kultur og informasjon,2020,"[2020/16894, 16894/2020, 20/16894, 16894/20]",Tsjekkia - kultur og informasjon,2020/16894,http://data.einnsyn.no/noark4/Saksmappe--97241...,[Saksmappe],16894,2022-03-29T08:56:45.233Z,2022-03-22,[Journalpost],Utenriksdepartementet,[iLiterature - Czech Republic],2022-03-22T00:00:00,2022-01-09,http://www.arkivverket.no/standarder/noark5/ar...,b0,[Skjerming],-,http://data.einnsyn.no/noark4/Journalpost--972...,[],[Utenriksdepartementet],[],Support - iLiteratura 2022,[iLiterature - Czech Republic],"[2020/16894-19, 16894/2020-19, 20/16894-19, 16...",Support - iLiteratura 2022,[],[{'korrespondansepartNavn': 'iLiterature - Cze...,[http://data.einnsyn.no/virksomhet/oepeksport_...,19,28236,http://www.arkivverket.no/standarder/noark5/ar...,2022,http://data.einnsyn.no/virksomhet/oepeksport_c...,20/16894-19,2022-01-09,2022-01-09,2022-03-22,2022-03-29 08:56:45.233000+00:00,Support - iLiteratura 2022,Tsjekkia - kultur og informasjon,Journalpost
1,0.0,http://data.einnsyn.no/noark4/Journalpost--972...,C-653/21 - Syndicat Uniclima,2021,"[2021/16707, 16707/2021, 21/16707, 16707/21]",C-653/21 - Syndicat Uniclima,2021/16707,http://data.einnsyn.no/noark4/Saksmappe--97241...,[Saksmappe],16707,2022-03-28T10:35:58.014Z,2022-03-21,[Journalpost],Utenriksdepartementet,[],2022-03-21T00:00:00,2022-01-07,http://www.arkivverket.no/standarder/noark5/ar...,b0,[Skjerming],"Offl. § 20, 1. ledd bokstav b)",http://data.einnsyn.no/noark4/Journalpost--972...,[Arbeids- og sosialdepartementet],[Utenriksdepartementet],[],C-653/21 - Syndicat Uniclima. Maskindirektivet...,[],"[2021/16707-7, 16707/2021-7, 21/16707-7, 16707...",C-653/21 - Syndicat Uniclima. Maskindirektivet...,[Arbeids- og sosialdepartementet],[{'korrespondansepartNavn': 'Arbeids- og sosia...,[http://data.einnsyn.no/virksomhet/oepeksport_...,7,30141,http://www.arkivverket.no/standarder/noark5/ar...,2022,http://data.einnsyn.no/virksomhet/oepeksport_c...,21/16707-7,2022-01-07,2022-01-07,2022-03-21,2022-03-28 10:35:58.014000+00:00,C-653/21 - Syndicat Uniclima. Maskindirektivet...,C-653/21 - Syndicat Uniclima,Journalpost
2,0.0,http://data.einnsyn.no/noark4/Journalpost--972...,Afghanistan - Møtene med Taliban i Oslo januar...,2022,"[2022/1105, 1105/2022, 22/1105, 1105/22]",Afghanistan - Møtene med Taliban i Oslo januar...,2022/1105,http://data.einnsyn.no/noark4/Saksmappe--97241...,[Saksmappe],1105,2022-03-25T07:05:33.505Z,2022-03-18,[Journalpost],Utenriksdepartementet,[Helse- og omsorgsdepartementet],2022-03-18T00:00:00,2022-01-10,http://www.arkivverket.no/standarder/noark5/ar...,b0,[Skjerming],-,http://data.einnsyn.no/noark4/Journalpost--972...,[],[Utenriksdepartementet],[],Rådføring om smittevernstiltak,[Helse- og omsorgsdepartementet],"[2022/1105-119, 1105/2022-119, 22/1105-119, 11...",Rådføring om smittevernstiltak,[],[{'korrespondansepartNavn': 'Helse- og omsorgs...,[http://data.einnsyn.no/virksomhet/oepeksport_...,119,22949,http://www.arkivverket.no/standarder/noark5/a

In [ ]:
df